In [1]:
from oltk.dataset import VaultDataset
from oltk.features import extract_header_props

I personally do not own an obsidian grapth big enough to check certain features.
I do not own any rights on the repository and it's contents by any means, the following data is in full posession of it's rightful author [Luke Tianpei](https://github.com/TianpeiLuke)

In [2]:
!mkdir ./tmp_data
%cd ./tmp_data
!git clone https://github.com/TianpeiLuke/SelfStudyNotes
%cd ../

/root/igkh/codes/ObsidianLTK/examples/tmp_data
Cloning into 'SelfStudyNotes'...
remote: Enumerating objects: 28898, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 28898 (delta 325), reused 324 (delta 324), pack-reused 28558 (from 2)
Receiving objects: 100% (28898/28898), 397.54 MiB | 6.96 MiB/s, done.
Resolving deltas: 100% (21018/21018), done.
Updating files: 100% (6492/6492), done.
/root/igkh/codes/ObsidianLTK/examples


In [19]:
vault = VaultDataset("./tmp_data/SelfStudyNotes/obsidian_notes")

In [20]:
%%time
vault.construct()

CPU times: user 488 ms, sys: 75.3 ms, total: 564 ms
Wall time: 543 ms


In [28]:
%%time
samples = vault.get_v()

CPU times: user 24 μs, sys: 3 μs, total: 27 μs
Wall time: 30 μs


In [29]:
from oltk.utils.re_constants import JINJA_EXPR, TEMPLATER_EXPR
import re
import polars as pl

In [30]:
def tidy_up_template(content: str) -> str:
    content = re.sub(JINJA_EXPR, "__TEMPLATE_IMPUTATION__", content)
    content = re.sub(TEMPLATER_EXPR, "__TEMPLATE_IMPUTATION__", content)
    return content

In [31]:
samples = samples.with_columns(
    pl.col("text")
    .map_elements(
        tidy_up_template,
        return_dtype=str,
    )
    .alias("text_without_templates")
).drop(pl.col("text"))

In [32]:
def extract_tags_and_date(x):
    props = extract_header_props(x)
    return {"tags": props.get("tags", []), "date": str(props.get("date of note", ""))}


In [33]:
samples = samples.with_columns(
    pl.col("text_without_templates")
    .map_elements(
        lambda x: extract_tags_and_date(x),
        return_dtype=pl.Struct({"tags": pl.List(str), "date": str}),
    )
    .alias("stat")
)

In [34]:
samples

id,title,text_without_templates,stat
u32,str,str,struct[2]
0,"""Talk with my girl""","""--- tags: - restaurant regio…","{[""restaurant""],""2025-07-24""}"
1,"""Skin on Leg Arm and Food""","""--- tags: - health - skin …","{[""health"", ""skin""],""2025-02-02""}"
2,"""Skin on Head""","""--- tags: - health - skin …","{[""health"", ""skin""],""2025-02-02""}"
3,"""Flowers""","""--- tags: aliases: date of n…","{null,""2024-12-23""}"
4,"""Chocolate Box Godvia""","""--- tags: aliases: date of n…","{null,""2025-02-05""}"
…,…,…,…
4177,"""Blog Snippet Card""","""--- tags: - excerpt aliases:…","{[""excerpt""],""__TEMPLATE_IMPUTATION__""}"
4178,"""Literature Summary Note Card""","""--- tags: - summary aliases…","{[""summary""],""__TEMPLATE_IMPUTATION__""}"
4179,"""Literature Reading Question Ca…","""--- tags: - critical_thinki…","{[""critical_thinking/raise_questions""],""__TEMPLATE_IMPUTATION__""}"


In [35]:
instances = samples.select("stat").unnest("stat").explode("tags").with_columns(pl.col("date").str.to_date(strict=False).alias("inferred_date")).drop("date")

In [43]:
instances

tags,inferred_date
str,date
"""restaurant""",2025-07-24
"""health""",2025-02-02
"""skin""",2025-02-02
"""health""",2025-02-02
"""skin""",2025-02-02
…,…
"""summary""",null
"""critical_thinking/raise_questi…",null
"""model""",null


In [42]:
len(instances["tags"].unique())

1295

In [37]:
from oltk.iziviz import progress_web

In [44]:
progress_web(instances, "tags", "inferred_date", ts_delta="2w", groups=["model", ["code", "code_snipet"], "critical_thinking"]).show()

In [ ]:
!rm -rf ./tmp_data